In [31]:
import API_llm as API

In [32]:
message = "Hello, this is a message!\n"
with open('output.txt', 'w') as file:
    file.write(message)
    # Erase the file content
    
    
def log(message):
    with open('output.txt', 'a') as file:
        file.write(message)
        file.write('\n')

In [33]:
def addHistory(players, message):
    for player in players:
        player.addHistory({"role": "system", "content": message})

In [34]:
import game_control as ll
# import easygui

players = [ll.player('玩家一'), ll.player('玩家二'), ll.player('玩家三')]
# easygui.msgbox(gameStartMessage, title="Simple GUI")
players[0].addHistory({"role": "system", "content": "你是一个斗地主机器人。"})
players[1].addHistory({"role": "system", "content": "你是一个斗地主机器人。"})
players[2].addHistory({"role": "system", "content": "你是一个斗地主机器人。"})


landLordNumber, gameStartMessage = ll.landlordDecide(players, ll.gameStart(players))
log(gameStartMessage)
addHistory(players, gameStartMessage)   


playing = landLordNumber
table = ll.series()
passCount = 0
gameEnd = 0

In [35]:
error_count = 0
turn_count = 0

while gameEnd == 0:
    # If everyone passed, clear the table
    if passCount == 2:
        table = ll.series()
        passCount = 0
        
    
    # Player's turn
    while True:
        # Tell the player the current table and player's hand
        current_message = players[playing].playerTurn(table)
        
        # Get response from the player
        response = API.get_llm_reaction(players[playing].getHistory(), current_message)
        players[playing].addHistory({"role": "assistant", "content": response})
        
        # Check if the player wants to pass
        if (response == 'PASS' or response == '不出') and table.seriesCards != []:
            passCount += 1
            log(players[playing].name + ' 不要')
            break
        
        # Check if the player cast a valid card
        playerResponseSeries, playerResponseCards = players[playing].cardsValid(response,table)
        
        verify, message = playerResponseSeries.compare(table)
        if verify:
            addHistory(players, players[playing].name + ' 出牌: ' + str(playerResponseSeries))
            log(players[playing].name + ' 出牌: ' + str(playerResponseSeries))
            table = playerResponseSeries
            playerOutput = playerResponseSeries
            players[playing].castCards(playerResponseCards)
            passCount = 0
            break
        
        elif message == '违规':
            players[playing].addHistory({"role": "system", "content": '你的出牌不符合规则, 请重新出牌'})
            error_count += 1
            continue
        else:
            players[playing].addHistory({"role": "system", "content": '上家比你牌大, 请重新出牌'})
            error_count += 1
            continue
        
    turn_count += 1
    
    # Check if the player has won
    gameEnd, winner = ll.gameEnd(players)
    if gameEnd == 1:
        log(players[winner].name + ' 赢了')
        break
    
    playing = (playing + 1) % 3
        
print('Error rate: ' + str(error_count/turn_count))
print('Turn count: ' + str(turn_count))

Error rate: 0.2872340425531915
Turn count: 94


In [36]:
for player in players:
    log(player.name + '的历史记录:')
    log('--------------------------------')
    for history in player.getHistory():
        log(str(history))